In [ ]:
import pandas as pd
from os import listdir

In [ ]:
def csv_to_X_y(path):
    folder = listdir(path)
    X = []
    y = []
    for index, file in enumerate(folder):
        df = pd.read_csv(f"{path}/{file}", header = None)
        X.extend(df.values)
        y.extend([index]*len(df))
    return X, y

In [ ]:
X, y = csv_to_X_y('../test_features')

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming X is your feature matrix and y is the corresponding labels

# Multiclass SVM with Gaussian kernel and 10-fold cross-validation
svm_classifier = SVC(kernel='rbf', gamma='scale', C=1.0)  # You can adjust kernel parameters as needed

# Using stratified K-fold for multiclass classification
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

svm_scores = cross_val_score(svm_classifier, X, y, cv=kf)
print("Multiclass SVM Accuracy:", np.mean(svm_scores))

# LDA classifier with Monte-Carlo cross-validation (50 runs)
lda_classifier = LinearDiscriminantAnalysis()

# Assuming you have multiple runs and want to average the results
num_runs = 50
lda_accuracies = []

for run in range(num_runs):
    # Split the data into training and testing sets for each run
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=run)

    lda_classifier.fit(X_train, y_train)
    y_pred = lda_classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    lda_accuracies.append(accuracy)

average_lda_accuracy = np.mean(lda_accuracies)
print("LDA Average Accuracy over 50 runs:", average_lda_accuracy)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris

# Load the Iris dataset as an example
# iris = load_iris()
# X = iris.data
# y = iris.target

# Standardize the data (optional, but recommended for PCA)
# mean = np.mean(X, axis=0)
# std = np.std(X, axis=0)
# X_standardized = (X - mean) / std

# Perform PCA
pca = PCA()
X_pca = pca.fit_transform(X)

# Scree plot
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# Plotting
plt.figure(figsize=(10, 6))

# Scree plot
plt.subplot(1, 2, 1)
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')

# Cumulative explained variance
plt.subplot(1, 2, 2)
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Variance Explained')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import load_iris

# # Load the Iris dataset as an example
# iris = load_iris()
# X = iris.data
# y = iris.target

# Perform LDA
lda = LinearDiscriminantAnalysis()
X_lda = lda.fit_transform(X, y)

# Scree plot
explained_variance_ratio = lda.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# Plotting
plt.figure(figsize=(10, 6))

# Scree plot
plt.subplot(1, 2, 1)
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio)
plt.title('Scree Plot')
plt.xlabel('Linear Discriminant')
plt.ylabel('Variance Explained')

# Cumulative explained variance
plt.subplot(1, 2, 2)
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Linear Discriminants')
plt.ylabel('Cumulative Variance Explained')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

classes = ["granade", "gun 1", "gun 2", "jump", "punch"]

for i in range(5):
    df = pd.read_csv(f"../data/{classes[i]}/005.csv", header = None)
    
    x = list(range(len(df)))
    for j in range(3):
        plt.subplot(5, 1, i + 1)
        plt.plot(x, list(df.iloc[:, j]))
    plt.title(classes[i])

plt.show()

## Features

In [ ]:
import numpy as np
import scipy.stats as stats
from scipy.signal import find_peaks
import pandas as pd

fs = 100

def filtered(df):
    fft_result = np.fft.fft(df.values)
    fft_freq = np.fft.fftfreq(len(fft_result), 1/fs)

    # Identify low-frequency components
    low_freq_mask = np.abs(fft_freq) <= 4  # Change the threshold as needed
    low_freq_components = fft_result.copy()
    low_freq_components[~low_freq_mask] = 0

    # Inverse FFT to obtain the signal with only low-frequency components
    filtered_signal = np.fft.ifft(low_freq_components).real

    return filtered_signal

def correlation(df, index, rows):
    x = filtered(df.iloc[:, index])
    y = filtered(df.iloc[:, (index + 1)%rows])
    return np.corrcoef(x, y)[0, 1]

def features(df):
    X = [["max m"], ["min m"], ["imag"], ["imag mean"], ["100 th percentile"], ["iqr"], ["range"], ["std"], ["half mean"], ["half median"], ["half 70th percentile"], ["correl"]]
    for i in range(len(df.T)):
        fft_result = np.fft.fft(df.iloc[:, i].values)
        fft_freq = np.fft.fftfreq(len(fft_result), 1/fs)

        # Identify low-frequency components
        low_freq_mask = np.abs(fft_freq) <= 4  # Change the threshold as needed
        low_freq_components = fft_result.copy()
        low_freq_components[~low_freq_mask] = 0

        # Inverse FFT to obtain the signal with only low-frequency components
        filtered_signal = np.fft.ifft(low_freq_components).real
        
        peaks, _ = find_peaks(filtered_signal)
        # plt.plot(peaks, filtered_signal[peaks], "rx")
        valleys, _ = find_peaks(-filtered_signal)
        # plt.plot(valleys, filtered_signal[valleys], "bx") 

        dx = peaks[:, np.newaxis] - valleys
        dy = filtered_signal[peaks][:, np.newaxis] - filtered_signal[valleys]

        df_dx = (dy/dx).flatten()

        X[0].append([max(df_dx) if len(df_dx) != 0 else 0])
        X[1].append([min(df_dx) if len(df_dx) != 0 else 0])
        X[2].append([low_freq_components[1].imag])
        X[3].append([(low_freq_components[:3].imag).mean()])
        X[4].append([np.percentile(filtered_signal, 100)])
        X[5].append([stats.iqr(filtered_signal)])
        X[6].append([np.ptp(filtered_signal)])
        X[7].append([np.std(filtered_signal)])
        X[8].append([np.mean(filtered_signal[:25])])
        X[9].append([np.median(filtered_signal[:25])])
        X[10].append([np.percentile(filtered_signal[:25], 70)])
        X[11].append([correlation(df, i, len(df.T))])

    features = []

    for axis in X:
        features.extend(axis[1:])
    
    return features

## Selected Feature

In [ ]:
import numpy as np
import scipy.stats as stats
from scipy.signal import find_peaks
import pandas as pd

def filtered(df):
    data = []
    imaginary = []
    for i in range(3):
        fft_result = np.fft.fft(df.iloc[:, i].values)
        fft_freq = np.fft.fftfreq(len(fft_result), 0.01)

        # Identify low-frequency components
        low_freq_mask = np.abs(fft_freq) <= 4  # Change the threshold as needed
        low_freq_components = fft_result.copy()
        low_freq_components[~low_freq_mask] = 0

        # Inverse FFT to obtain the signal with only low-frequency components
        filtered_signal = np.fft.ifft(low_freq_components).real
        data.append(filtered_signal)
        imaginary.append(low_freq_components.imag[:3])
    return np.array(data), np.array(imaginary)

def slope(data):
    peaks, _ = find_peaks(data)
    # plt.plot(peaks, filtered_signal[peaks], "rx")
    valleys, _ = find_peaks(-data)
    # plt.plot(valleys, filtered_signal[valleys], "bx") 

    dx = peaks[:, np.newaxis] - valleys
    dy = data[peaks][:, np.newaxis] - data[valleys]

    return((dy/dx).flatten())

def accl_features(df):
    filtered_signal, _ = filtered(df)
    features = []

    df_dx = slope(filtered_signal[1])
    features.append([min(df_dx) if len(df_dx) != 0 else 0])

    df_dx = slope(filtered_signal[2])
    features.append([min(df_dx) if len(df_dx) != 0 else 0])

    features.append([np.percentile(filtered_signal[0], 100)])
    features.append([np.percentile(filtered_signal[1], 100)])

    features.append([stats.iqr(filtered_signal[1])])

    features.append([np.mean(filtered_signal[0][:25])])
    features.append([np.mean(filtered_signal[1][:25])])

    features.append([np.median(filtered_signal[1][:25])])

    features.append([np.percentile(filtered_signal[1][:25], 70)])
    return features

def gyro_features(df):
    filtered_signal, imaginary = filtered(df)
    features = []

    features.append([imaginary[0][1]])
    features.append([imaginary[2][1]])

    features.append([(imaginary[0]).mean()])
    features.append([(imaginary[2]).mean()])

    features.append([stats.iqr(filtered_signal[0])])
    features.append([stats.iqr(filtered_signal[2])])

    features.append([np.ptp(filtered_signal[2])])

    features.append([np.std(filtered_signal[2])])

    features.append([np.mean(filtered_signal[0][:25])])

    features.append([np.median(filtered_signal[0][:25])])
    return features

## Feature Extractor

In [ ]:
from os import listdir
import pandas as pd

folders = listdir("../data")

columns = [(5, 55), (20, 70), (10, 60), (15, 65), (5, 55), (5, 55)]

for index, folder in enumerate(folders):
    open(f"../test_features/{folder}.csv", "w").close()

    files = listdir(f"../data/{folder}")[:17]
    
    for file in files:
        data = pd.read_csv(f"../data/{folder}/{file}", header=None).iloc[columns[index][0]:columns[index][1], :6]
        # feature = features(data.iloc[:, 3:6])
        # feature = features(data.iloc[:, 0:3])
        # feature.extend(features(data.iloc[:, 3:6]))

        feature = accl_features(data.iloc[:, 0:3])
        feature.extend(gyro_features(data.iloc[:, 3:6]))

        pd.DataFrame(feature).T.to_csv(f"../test_features/{folder}.csv", header = None, index = False, mode = "a")

## Test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os import listdir
from scipy.signal import find_peaks

folders = listdir("../data")[:-1]
columns = [(5, 55), (20, 70), (10, 60), (15, 65), (5, 55)]
fs = 100
signals = []

for folder in folders:
    files = listdir(f"../data/{folder}")[:17]
    signal = [pd.read_csv(f"../data/{folder}/{file}") for file in files]
    signals.append(signal)

for i in range(5):
    plt.figure(figsize = (8,7))
    
    for j in range(5):
        for k in range(3):
            signal = signals[j][i].iloc[columns[j][0]:columns[j][1], k].values
            # signal -= min(signal)
            # signal /= np.ptp(signal)

            fft_result = np.fft.fft(signal)
            fft_freq = np.fft.fftfreq(len(fft_result), 1/fs)

            # Identify low-frequency components
            low_freq_mask = np.abs(fft_freq) == 2  # Change the threshold as needed
            low_freq_components = fft_result.copy()
            low_freq_components[~low_freq_mask] = 0

            # Inverse FFT to obtain the signal with only low-frequency components
            filtered_signal = np.fft.ifft(low_freq_components).real
            plt.subplot(5, 1, j + 1)
            # plt.plot(signal, alpha = 0.6)
            plt.plot(filtered_signal)

            # peaks, _ = find_peaks(filtered_signal, distance = 20, prominence = 0.2)
            # plt.plot(peaks, filtered_signal[peaks], "rx")
            # valleys, _ = find_peaks(-filtered_signal)
            # plt.plot(valleys, filtered_signal[valleys], "bx")
            
            # dx = peaks[:, np.newaxis] - valleys
            # dy = filtered_signal[peaks][:, np.newaxis] - filtered_signal[valleys]
            # # print(max(abs((dy/dx).flatten())))
            # # print(dy/dx)
            # plt.text(55, (2 - k)*5000, str(max(abs((dy/dx).flatten()))))

    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os import listdir
from scipy.signal import find_peaks
import scipy.stats as stats

folders = listdir("../data")[:-1]
columns = [(5, 55), (20, 70), (10, 60), (15, 65), (5, 55)]
fs = 100
signals = []

for folder in folders:
    files = listdir(f"../data/{folder}")[:17]
    signal = [pd.read_csv(f"../data/{folder}/{file}") for file in files]
    signals.append(signal)

Mean = []

peak = []
valley = []
dydx = []

for i in range(17):
    mean1 = []
    for j in range(5):
        mean2 = []
        for k in range(3):
            signal = signals[j][i].iloc[columns[j][0]:columns[j][1], k].values
            # signal -= min(signal)
            # signal /= np.ptp(signal)

            fft_result = np.fft.fft(signal)
            fft_freq = np.fft.fftfreq(len(fft_result), 1/fs)

            # Identify low-frequency components
            low_freq_mask = np.abs(fft_freq) <= 4  # Change the threshold as needed
            low_freq_components = fft_result.copy()
            low_freq_components[~low_freq_mask] = 0

            # Inverse FFT to obtain the signal with only low-frequency components
            filtered_signal = np.fft.ifft(low_freq_components).real
            
            peaks, _ = find_peaks(filtered_signal)
            # plt.plot(peaks, filtered_signal[peaks], "rx")
            valleys, _ = find_peaks(-filtered_signal)
            # plt.plot(valleys, filtered_signal[valleys], "bx")
            
            peak.append(peaks)
            valley.append(valleys)

            dx = peaks[:, np.newaxis] - valleys
            dy = filtered_signal[peaks][:, np.newaxis] - filtered_signal[valleys]

            dydx.append(dy/dx)
            
            mean2.append([max((dy/dx).flatten())])
            # mean2.append([min((dy/dx).flatten())])
            # mean2.append([low_freq_components[1].imag])
            # mean2.append([(low_freq_components[:3].imag).mean()])
            # mean2.append([np.percentile(filtered_signal, 100)])
            # mean2.append([stats.iqr(filtered_signal)])
            # mean2.append([np.ptp(filtered_signal)])
            # mean2.append([np.std(filtered_signal)])
            # mean2.append([np.mean(filtered_signal[:25])])
            # mean2.append([np.median(filtered_signal[:25])])
            # mean2.append([np.percentile(filtered_signal[:25], 70)])
        mean1.append(mean2)
    Mean.append(mean1)

for i in range(3):
    for j in range(5):
        for k in range(17):
            plt.scatter(Mean[k][j][i], j)
    plt.show()

## speed graph calculator

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(13, 8))

Cm = 0.95
g = 9.81
M = 200
Cwa = 1.2
A = 1
roh = 1.3
P = 1500

w = 0/3.6

Crs = list(np.linspace(0.001, 0.3, 100))

for theta in range(0, 61, 5):
    theta = np.deg2rad(theta)
    v = []
    for Cr in Crs:
        Fr = Cr*M*g*np.cos(theta)
        Fg = M*g*np.sin(theta)

        Fstg = Fr + Fg

        a = 1
        b = 3*w
        c = (b*w) + (2*(Fstg)/Cwa*A*roh)
        d = w**3 - (2*P*Cm/Cwa*A*roh)

        v.append([max(np.roots([a, b, c, d]).real)*3.6])
    plt.plot(Crs, v, label = str(np.round(np.rad2deg(theta), 0)))

plt.title(f"Power {P}W, Headwind speed {w*3.6}Km/hr, Total Mass {M}Kg")
plt.grid(True)
plt.legend(title = "Terrain slope")
plt.yticks(range(0, 51, 5))
plt.ylabel("velocity (Km/hr)")
plt.xticks(np.round(np.linspace(0.001, 0.3, 15), 3))
plt.xlabel("Cr\n0.011 for asphalt, 0.02 for dirt road, 0.3 for farm land")
plt.show()    

## power calculator

In [ ]:
Cm = 0.95
v = 8.2/3.6
g = 9.81    
Cr = 0.3
theta = np.deg2rad(0)
M = 200
Cwa = 1.2
A = 1
roh = 1.2
w = 10/3.6

Fr = Cr*M*g*np.cos(theta)
Fg = M*g*np.sin(theta)

Fstg = Fr + Fg

Pag = 0.5*Cwa*A*roh*(v + w)**3

Power = ((Fstg*v) + (Pag))/Cm
print(Power)